# Model 4 from report - CNN without trainable embeddings: This models consists of a Convolutional Neural Network (CNN) with window size 4,  without updating the embeddings during training.

### The graph is shown below 
![graph can't be found](../data/images/graph_cnn.png)

In [2]:
from keras.layers import Embedding
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras import layers
from keras import backend as K
import keras
import tensorflow as tf

import logger
from logger import NBatchLogger



In [2]:
trainX,trainY,testX,testY,X_test,num_vocab, features=pickle.load(open('train_test_prep_full','rb'))

In [3]:
trainX=np.concatenate([trainX,testX[:485000]])
trainY=np.concatenate([trainY,testY[:485000]])
testX=testX[485000:]
testY=testY[485000:]

In [4]:
embeddings_index = {}
f = open( './twitter-datasets/glove.6B/glove.6B.200d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float64')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [5]:
embedding_matrix = np.zeros((num_vocab,200))
        
for i,feature in enumerate(features):
    embedding_vector=embeddings_index.get(feature)
    if(embedding_vector is not None):
        embedding_matrix[i] = embedding_vector
del(embeddings_index) # free up memory 

In [6]:
keras.backend.clear_session()
embedding_layer = Embedding(num_vocab,
                            200,
                            weights=[embedding_matrix],
                            input_length=100,
                            trainable=False,name='embedding1')
embedding_layer_2 = Embedding(num_vocab,
                            200,
                            weights=[embedding_matrix],
                            input_length=100,
                            trainable=False,name='embedding2')
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True,\
                                          embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=features)

In [3]:
callback_save=NBatchLogger('cnn_fix_embed_1',testX,testY,display=1000,if_exist='replace')

NameError: name 'testX' is not defined

In [10]:

callback_save=NBatchLogger('cnn_fix_embed_1',testX,testY,display=1000,if_exist='replace')

sequence_input = layers.Input(shape=(100,), dtype='int32',name='main_input')
embedded_sequences = embedding_layer(sequence_input)
net1 = Conv1D(128, 4, activation='relu',name = 'conv1_1')(embedded_sequences)
net1 = layers.Dropout(0.2,name = 'dropout1_1')(net1)
net1 = MaxPooling1D(pool_size=4, strides=2,name = 'max1_1')(net1)
net1 = Conv1D(128, 4, activation='relu',name = 'conv1_2')(net1)
net1 = layers.Dropout(0.1,name = 'dropout1_2')(net1)
net1 = MaxPooling1D(4,name = 'max1_2')(net1)
net1 = Conv1D(128, 4, activation='relu',name = 'conv1_3')(net1)
net1 = layers.Dropout(0.1,name = 'dropout1_3')(net1)
net1 = Flatten()(net1)
net1 = Dense(128, activation='relu',name = 'dense1_1')(net1)


In [11]:
'''
sequence_input_2 = layers.Input(shape=(100,), dtype='int32',name='main_input2')
net2 = embedding_layer(sequence_input_2)
net2 = Bidirectional(LSTM(256,return_sequences=True),name= 'bidir_LSTM')(net2)
net2 = layers.LSTM(64,name='LSTM_2')(net2)
net2 = layers.Dropout(0.3,name='dropout2_1')(net2)
net2 = Dense(128, activation='relu',name='dense2_1')(net2)
'''

"\nsequence_input_2 = layers.Input(shape=(100,), dtype='int32',name='main_input2')\nnet2 = embedding_layer(sequence_input_2)\nnet2 = Bidirectional(LSTM(256,return_sequences=True),name= 'bidir_LSTM')(net2)\nnet2 = layers.LSTM(64,name='LSTM_2')(net2)\nnet2 = layers.Dropout(0.3,name='dropout2_1')(net2)\nnet2 = Dense(128, activation='relu',name='dense2_1')(net2)\n"

In [12]:
#merged = layers.add(net1,name='merged')

In [14]:
merged = layers.Dropout(0.3)(net1)
merged = Dense(128, activation='relu')(merged)
preds = layers.Dense(2, activation='softmax')(merged)

model = keras.Model(sequence_input, preds)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])




In [16]:
print('Training')
model.fit(trainX, trainY,
          batch_size=64,
          epochs=10,
          validation_data=(testX,testY),callbacks=[tbCallBack,callback_save],verbose=1)

Training
Train on 2485000 samples, validate on 15000 samples
Epoch 1/10


FailedPreconditionError: Attempting to use uninitialized value embedding1/embeddings
	 [[Node: embedding1/embeddings/read = Identity[T=DT_FLOAT, _class=["loc:@embedding1/embeddings"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](embedding1/embeddings)]]
	 [[Node: loss/mul/_145 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1216_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'embedding1/embeddings/read', defined at:
  File "/home/keshav/anaconda3/envs/py35/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/keshav/anaconda3/envs/py35/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/keshav/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    